In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime

building_area_m2 = 8000
max_occupancy = 400

start_date = datetime(2022, 1, 1, 0, 0, 0)
end_date = datetime(2025, 12, 31, 23, 0, 0)
dates = pd.date_range(start=start_date, end=end_date, freq='h')

df = pd.DataFrame(index=dates)
df.index.name = 'Timestamp'

df['Hour'] = df.index.hour
df['Day_of_Week'] = df.index.dayofweek
df['Is_Weekend'] = (df['Day_of_Week'] >= 5).astype(int)
df['Is_Daytime'] = ((df['Hour'] >= 7) & (df['Hour'] <= 18)).astype(int)
df['Month'] = df.index.month
df['Season'] = ((df.index.month % 12 + 3) // 3).astype(int)

day_of_year = df.index.dayofyear
df['OutsideWeather_Temp_C'] = 22 + 12 * np.sin(2 * np.pi * (day_of_year - 80) / 365.25) + np.random.normal(0, 3, len(df))
df['OutsideWeather_Temp_C'] = df['OutsideWeather_Temp_C'].clip(5, 40)

df['OutsideWeather_Humidity_Pct'] = 60 - 25 * np.sin(2 * np.pi * (day_of_year - 80) / 365.25) + np.random.normal(0, 10, len(df))
df['OutsideWeather_Humidity_Pct'] = df['OutsideWeather_Humidity_Pct'].clip(20, 95)

df['Pressure_mmHg'] = np.random.normal(760, 8, len(df)).clip(740, 780)

df['Wind_Speed_m_s'] = np.random.exponential(4, len(df)).clip(0, 25)

hour_factor = np.where((df['Hour'] >= 8) & (df['Hour'] <= 17), 1.0, 0.3)
weekend_factor = np.where(df['Is_Weekend'] == 1, 0.15, 1.0)
season_factor = np.where(df['Season'] == 3, 1.1, 1.0)

base_occupancy = max_occupancy * hour_factor * weekend_factor * season_factor
df['Total_Occupancy_Count'] = np.round(base_occupancy + np.random.normal(0, 50, len(df))).clip(0, max_occupancy).astype(int)

df['Temp_Deviation'] = np.abs(df['OutsideWeather_Temp_C'] - 22)

df['HVAC_Load_Estimate'] = building_area_m2 * df['Temp_Deviation'] * 0.01 + df['Total_Occupancy_Count'] * 30

df['Temp_Occupancy_Interaction'] = df['Temp_Deviation'] * df['Total_Occupancy_Count']

df['Indoor_Temp_C'] = df['OutsideWeather_Temp_C'] + np.random.normal(0, 2, len(df))
df['Indoor_Temp_C'] = df['Indoor_Temp_C'].clip(18, 28)

df['Indoor_Humidity_Pct'] = 50 + np.random.normal(0, 5, len(df)).clip(30, 70)

df['Indoor_Temp_Deviation'] = np.abs(df['Indoor_Temp_C'] - 22)
df['Indoor_Humidity_Deviation'] = np.abs(df['Indoor_Humidity_Pct'] - 50)
df['Humidity_Deviation'] = np.abs(df['OutsideWeather_Humidity_Pct'] - 50)

df['Daylight_Hours_Factor'] = df['Is_Daytime'] * (0.4 - df['OutsideWeather_Temp_C'].clip(10, 30) / 100)
df['Lighting_Occupancy_Ratio'] = df['Total_Occupancy_Count'] / (max_occupancy + 1)
df['Solar_Irradiance_Estimate'] = np.maximum(0, np.sin(np.pi * (df['Hour'] - 6) / 12)) * (1 - df['Is_Weekend']) * 800

df['Plug_Peak_Hour'] = ((df['Hour'] >= 9) & (df['Hour'] <= 17)).astype(int)
df['Device_Usage_Factor'] = df['Total_Occupancy_Count'] / max_occupancy * (1 + 0.2 * (df['Season'] == 3))
df['Remote_Work_Factor'] = np.where(df['Is_Weekend'] == 1, 0.3, 1.0)

base_price = 0.10
peak_factor = np.where((df['Hour'] >= 12) & (df['Hour'] <= 20), 1.5, 1.0)
season_price = np.where(df['Season'] == 3, 1.3, 1.0)
df['Energy_Price_USD_kWh'] = base_price * peak_factor * season_price + np.random.normal(0, 0.01, len(df))

df['Price_Sensitivity'] = np.where(df['Energy_Price_USD_kWh'] > 0.15, 0.9, 1.0)

df['Energy_HVAC_Wh'] = df['HVAC_Load_Estimate'] * 60 + df['Total_Occupancy_Count'] * 40 + np.random.normal(0, 12000, len(df))
df['Energy_HVAC_Wh'] = df['Energy_HVAC_Wh'].clip(lower=40000)

lighting_base = building_area_m2 * 10
daylight_factor = 1 - df['Is_Daytime'] * 0.35 - df['Solar_Irradiance_Estimate'] / 5000
occupancy_factor = df['Lighting_Occupancy_Ratio']
df['Energy_Lighting_Wh'] = lighting_base * daylight_factor * occupancy_factor * (1 + np.random.normal(0, 0.35, len(df)))
df['Energy_Lighting_Wh'] = df['Energy_Lighting_Wh'].clip(lower=5000)  # lower أقل عشان no spike

df['Energy_Plug_Wh'] = df['Total_Occupancy_Count'] * 100 * df['Device_Usage_Factor'] * df['Remote_Work_Factor'] * df['Price_Sensitivity'] + building_area_m2 * 5 + np.random.normal(0, 8000, len(df))
df['Energy_Plug_Wh'] = df['Energy_Plug_Wh'].clip(lower=15000)

df['Energy_Other_Wh'] = building_area_m2 * 10 + np.random.normal(0, 8000, len(df))
df['Energy_Other_Wh'] = df['Energy_Other_Wh'].clip(lower=25000)

df['Total_Energy_Wh'] = df['Energy_HVAC_Wh'] + df['Energy_Lighting_Wh'] + df['Energy_Plug_Wh'] + df['Energy_Other_Wh']

df['Building_Area_m2'] = building_area_m2

output_path = r"C:\Users\Laptop World\Building_Energy_Twin_Sequential_3Years.csv"
df.to_csv(output_path)
